Implement a simple Moving Average algorithm on the collected stock price data to kick-start your implementation strategy.

In [1]:
import pandas as pd

def generate_signals(data, short_window=20, long_window=50):
    """
    Generates buy/sell signals based on SMA Crossover.
    'data' should be a DataFrame with a 'Close' price column.
    """
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    # calulate moving averages
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

    # 1.0 when short is above long, else 0.0
    signals['signal'] = 0.0
    signals['signal'][short_window:] = np.where(
        signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0
    )


    # 1.0 = Buy, -1.0 = Sell
    signals['positions'] = signals['signal'].diff()

    return signals

Discuss more advanced algorithms: ARIMA, LSTM, etc. Explore evaluation metrics for time series data: Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), etc.

Select an algorithm or develop a hybrid algorithm to predict the stock prices and provide buy/sell
signals.
Implement the chosen algorithm(s) in Python. Create functions that generate buy/sell signals based
on the stock price data

In [ ]:
import pandas as pd
import numpy as np

def generate_hybrid_signals(data, short_window=20, long_window=50, rsi_period=14):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    # moving averages
    signals['short_mavg'] = data['Close'].rolling(window=short_window).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window).mean()

    # rsi calculations
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    signals['rsi'] = 100 - (100 / (1 + rs))

    # hybrid logic
    signals['signal'] = 0.0
    signals['signal'] = np.where(
        (signals['short_mavg'] > signals['long_mavg']) & (signals['rsi'] < 70), 1.0, 0.0
    )

    signals['positions'] = signals['signal'].diff()
    return signals

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# remove NaNs
valid_data = signals.dropna()

# eval metrics
mae = mean_absolute_error(valid_data['price'], valid_data['short_mavg'])
rmse = np.sqrt(mean_squared_error(valid_data['price'], valid_data['short_mavg']))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")